## Learning Convolutional Kernels with NNSP

In [1]:
import numpy as np
from scipy.optimize import nnls

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def A_transpose_r_2d(data, r):
    n, t = data.shape
    Ar = np.zeros(t - 1)
    for i in range(t - 1):
        ai = np.zeros(n * t)
        for j in range(i + 1):
            ai[n * j : n * (j + 1)] = data[:, t + j - i - 1]
        for k in range(i + 1, t):
            ai[n * k : n * (k + 1)] = data[:, k - i - 1]
        Ar[i] = np.inner(ai, r)
    return Ar

def A_ind_S_2d(data, S):
    n, t = data.shape
    s = S.shape[0]
    AS = np.zeros((n * t, s))
    pi = 0
    for i in S:
        ai = np.zeros(n * t)
        for j in range(i + 1):
            ai[n * j : n * (j + 1)] = data[:, t + j - i - 1]
        for k in range(i + 1, t):
            ai[n * k : n * (k + 1)] = data[:, k - i - 1]
        AS[:, pi] = ai
        pi += 1
    return AS

def A_transpose_r_3d(data, r):
    m, n, t = data.shape
    Ar = np.zeros(t - 1)
    for i in range(t - 1):
        ai = np.zeros(m * n * t)
        for j in range(i + 1):
            ai[(m * n) * j : (m * n) * (j + 1)] = np.reshape(data[:, :, t + j - i - 1], -1, order = 'F')
        for k in range(i + 1, t):
            ai[(m * n) * k : (m * n) * (k + 1)] = np.reshape(data[:, :, k - i - 1], -1, order = 'F')
        Ar[i] = np.inner(ai, r)
    return Ar

def A_ind_S_3d(data, S):
    m, n, t = data.shape
    s = S.shape[0]
    AS = np.zeros((m * n * t, s))
    pi = 0
    for i in S:
        ai = np.zeros(m * n * t)
        for j in range(i + 1):
            ai[(m * n) * j : (m * n) * (j + 1)] = np.reshape(data[:, :, t + j - i - 1], -1, order = 'F')
        for k in range(i + 1, t):
            ai[(m * n) * k : (m * n) * (k + 1)] = np.reshape(data[:, :, k - i - 1], -1, order = 'F')
        AS[:, pi] = ai
        pi += 1
    return AS

def SP_ts(data, tau, stop = np.infty, nonnegative = True, type = 3, epsilon = 1e-2):
    if type == 3:
        m, n, t = data.shape
        x = np.reshape(ten2mat(data, 0), -1, order = 'F')
    elif type == 2:
        n, t = data.shape
        x = np.reshape(data, -1, order = 'F')
    r = x
    w = np.zeros(t - 1)
    S = np.array([])
    i = 0
    while np.linalg.norm(r, 2) > epsilon and i < stop:
        if type == 3:
            Ar = A_transpose_r_3d(data, r)
        elif type == 2:
            Ar = A_transpose_r_2d(data, r)
        S0 = np.argsort(abs(Ar))[- tau :]
        S = np.append(S[:], S0[:]).astype(int)
        if type == 3:
            AS = A_ind_S_3d(data, S)
        elif type == 2:
            AS = A_ind_S_2d(data, S)
        if nonnegative == True:
            w[S], _ = nnls(AS, x)
        elif nonnegative == False:
            w[S] = np.linalg.pinv(AS) @ x
        S = np.argsort(abs(w))[- tau :]
        w = np.zeros(t - 1)
        if type == 3:
            AS = A_ind_S_3d(data, S)
        elif type == 2:
            AS = A_ind_S_2d(data, S)
        if nonnegative == True:
            w[S], _ = nnls(AS, x)
        elif nonnegative == False:
            w[S] = np.linalg.pinv(AS) @ x
        r = x - AS @ w[S]
        i += 1
        print('Indices of non-zero coefficients (support set):', S)
        print('Non-zero entries of sparse temporal kernel:', w[S])
        print()
    return w, S, r

def OMP_ts(data, tau, stop = np.infty, nonnegative = True, type = 3, epsilon = 1e-2):
    if type == 3:
        m, n, t = data.shape
        x = np.reshape(ten2mat(data, 0), -1, order = 'F')
    elif type == 2:
        n, t = data.shape
        x = np.reshape(data, -1, order = 'F')
    r = x
    w = np.zeros(t - 1)
    S = np.array([])
    i = 0
    while np.linalg.norm(r, 2) > epsilon and i < stop and len(S) < tau:
        if type == 3:
            Ar = A_transpose_r_3d(data, r)
        elif type == 2:
            Ar = A_transpose_r_2d(data, r)
        if i > 1:
            Ar[S] = 0
        S0 = np.argmax(abs(Ar))
        S = np.append(S[:], S0).astype(int)
        if type == 3:
            AS = A_ind_S_3d(data, S)
        elif type == 2:
            AS = A_ind_S_2d(data, S)
        if nonnegative == True:
            w[S], _ = nnls(AS, x)
        elif nonnegative == False:
            w[S] = np.linalg.pinv(AS) @ x
        r = x - AS @ w[S]
        i += 1
        print('Indices of non-zero coefficients (support set):', S)
        print('Non-zero entries of sparse temporal kernel:', w[S])
        print()
    return w, S, r

## NYC Data

### NYC Ridesharing and Taxi in 2024

In [ ]:
import numpy as np
import time

tensor = np.load('../NYC-data/NYC_rideshare_mob_tensor_24.npz')['arr_0'][:, :, : 8 * 7 * 24]
tau = 4

start = time.time()
w, S, r = SP_ts(tensor, tau, 5, True)
end = time.time()
print('Indices of non-zero coefficients (support set):', S)
print('Non-zero entries of sparse temporal kernel:', w[S])
print('Loss function:', np.linalg.norm(r, 2) ** 2)
print('Running time (s):', end - start)

In [ ]:
import numpy as np
import time

tensor = np.load('../NYC-data/NYC_taxi_mob_tensor_24.npz')['arr_0'][:, :, : 8 * 7 * 24]
tau = 4

start = time.time()
w, S, r = SP_ts(tensor, tau, 5, True)
end = time.time()
print('Indices of non-zero coefficients (support set):', S)
print('Non-zero entries of sparse temporal kernel:', w[S])
print('Loss function:', np.linalg.norm(r, 2) ** 2)
print('Running time (s):', end - start)

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>